In [84]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
import matplotlib.pyplot as plt

from prophet import Prophet

from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier

import os
import re
import random
import logging
import warnings
from pathlib import Path

sns.set_style("whitegrid")

logging.getLogger("prophet").setLevel(logging.WARNING)
logging.getLogger("cmdstanpy").disabled=True

In [85]:
pd.ExcelFile('data/Datathon@MetuStatClub Final.xlsx').sheet_names

['Alım Davranışı Takip', 'Reklam Test Araştırması']

In [86]:
buying_behaviour_tracking = pd.read_excel("data/Datathon@MetuStatClub Final.xlsx", sheet_name="Alım Davranışı Takip")
ad_test_research = pd.read_excel("data/Datathon@MetuStatClub Final.xlsx", sheet_name="Reklam Test Araştırması")

In [87]:
buying_behaviour_tracking.describe()

,Company,Brand,Kategori,Metric Type,Metric,2020 Ocak,2020 Subat,2020 Mart,2020 Nisan,2020 Mayıs,...,2022 Mart,2022 Nisan,2022 Mayıs,2022 Haziran,2022 Temmuz,2022 Agustos,2022 Eylül,2022 Ekim,2022 Kasım,2022 Aralık
count,1550,1550,1550,1550,1550,1409,1409,1409,1409,1409,...,1420,1409,1409,1409,1409,1409,1409,1409,1409,1398
unique,11,69,5,2,23,744,744,742,746,740,...,756,748,742,744,725,738,730,739,745,733
top,ANZ,Mavi,3,Brand,Penetration,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
freq,703,44,506,790,70,68,63,66,99,98,...,34,72,67,72,71,70,72,37,35,37


In [88]:
# first row is total row, we don't need it
buying_behaviour_tracking = buying_behaviour_tracking.iloc[1:]

# change xxx to NaN
ad_test_research = ad_test_research.replace(r'^[xX]+$', np.nan, regex=True)

buying_behaviour_tracking["Company-Brand"] = buying_behaviour_tracking["Company"] + "-" + buying_behaviour_tracking["Brand"]

buying_behaviour_tracking.loc[buying_behaviour_tracking['Company'] == buying_behaviour_tracking['Brand'], 'Company-Brand'] = buying_behaviour_tracking['Company']
# drop company and brand columns
buying_behaviour_tracking = buying_behaviour_tracking.drop(["Company", "Brand"], axis=1)


In [89]:

df = buying_behaviour_tracking

df = pd.melt(
    df,
    id_vars=['Company-Brand', 'Metric', 'Kategori','Metric Type'],
    var_name='Dates',
    value_name='Value'
)

In [90]:
df.isna().sum()

Company-Brand       0
Metric              0
Kategori            0
Metric Type         0
Dates               0
Value            5054
dtype: int64

In [91]:
df = df.replace("-", np.nan)

In [92]:
df.isna().sum()

Company-Brand       0
Metric              0
Kategori            0
Metric Type         0
Dates               0
Value            7448
dtype: int64

In [93]:
df['Value'] = df.groupby('Metric')['Value'].transform(lambda x: x.fillna(x.mean()))
df['Value'] = df['Value'].round(2)
df

,Company-Brand,Metric,Kategori,Metric Type,Dates,Value
0,Prizy-Y,TOM,1,Brand,2020 Ocak,12.00
1,Prizy-Y,Spontaneous Awareness,1,Brand,2020 Ocak,33.00
2,Prizy-Y,Awareness,1,Brand,2020 Ocak,94.00
3,Prizy-Y,Consideration,1,Brand,2020 Ocak,93.00
4,Prizy-Y,Ever Used,1,Brand,2020 Ocak,87.26
...,...,...,...,...,...,...
55759,DLZ,PF (times),4,Behavior,2022 Aralık,1.27
55760,DLZ,TS (gr),4,Behavior,2022 Aralık,498.45
55761,DLZ,TV (tl),4,Behavior,2022 Aralık,17.65
55762,DLZ,Price (gr/tl),4,Behavior,2022 Aralık,0.04


In [94]:
translate_months = {
    'Ocak': 'January',
    'Şubat': 'February',
    'Subat': 'February',
    'Mart': 'March',
    'Nisan': 'April',
    'Mayıs': 'May',
    'Mayis': 'May',
    'Haziran': 'June',
    'Temmuz': 'July',
    'Ağustos': 'August',
    'Agustos': 'August',
    'Eylül': 'September',
    'Eylul': 'September',
    'Ekim': 'October',
    'Kasım': 'November',
    'Kasim': 'November',
    'Aralık': 'December',
    'Aralik': 'December'
}

df['Dates'] = df['Dates'].replace(translate_months, regex=True)
df['Dates'] = pd.to_datetime(df['Dates'], format='%Y %B')

In [95]:
df['Value'] = df['Value'].round(2)

In [96]:
pivoted = df.pivot(index=["Company-Brand",'Dates'], columns='Metric', values='Value')


pivoted["Market Share"] = pivoted["Volume (gr)"] / pivoted["Volume (gr)"].sum()
pivoted = pivoted.reset_index()
df2 = pd.melt(
    pivoted,
    id_vars=["Company-Brand",'Dates'],
    var_name="Metric",
    value_name="Value"
)
df2

,Company-Brand,Dates,Metric,Value
0,ANZ-A,2020-01-01,Awareness,98.000000
1,ANZ-A,2020-02-01,Awareness,97.000000
2,ANZ-A,2020-03-01,Awareness,96.000000
3,ANZ-A,2020-04-01,Awareness,98.000000
4,ANZ-A,2020-05-01,Awareness,97.000000
...,...,...,...,...
59611,Prizy-ZV,2022-08-01,Market Share,0.000031
59612,Prizy-ZV,2022-09-01,Market Share,0.000050
59613,Prizy-ZV,2022-10-01,Market Share,0.000042
59614,Prizy-ZV,2022-11-01,Market Share,0.000034


In [97]:
pivoted

Metric,Company-Brand,Dates,Awareness,Consideration,Ever Used,Most Often,No of Sku (barcode),PF (times),Penetration,Performance,...,Spontaneous Awareness,TOM,TS (gr),TV (tl),Total Ad Awareness (T5B),Trial,Value (tl),Volume (gr),Yakınlık,Market Share
0,ANZ-A,2020-01-01,98.00,95.00,87.26,44.00,80.0,1.46,4575322.59,63.00,...,86.0,45.00,88.16,5.05,NaN,85.00,33813115.02,5.902114e+08,73.00,0.001017
1,ANZ-A,2020-02-01,97.00,93.00,87.26,37.00,84.0,1.45,4943679.08,56.00,...,80.0,37.00,91.31,5.25,NaN,81.00,37605796.32,6.541825e+08,62.00,0.001127
2,ANZ-A,2020-03-01,96.00,89.00,87.26,30.00,85.0,1.43,4378076.07,50.00,...,82.0,36.00,92.27,5.35,NaN,75.00,33493507.49,5.773998e+08,55.00,0.000995
3,ANZ-A,2020-04-01,98.00,92.00,87.26,30.00,77.0,1.48,4461556.02,58.00,...,86.0,38.00,104.23,6.05,NaN,79.00,39896879.66,6.878893e+08,58.00,0.001185
4,ANZ-A,2020-05-01,97.00,92.00,87.26,32.00,77.0,1.44,3951388.79,60.00,...,88.0,36.00,107.47,6.25,NaN,79.00,35554699.71,6.108979e+08,57.00,0.001053
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2479,Prizy-ZV,2022-08-01,90.52,69.79,87.26,7.17,3.0,1.11,186480.05,44.12,...,4.0,0.00,86.89,9.49,36.46,62.53,1965684.04,1.799400e+07,42.11,0.000031
2480,Prizy-ZV,2022-09-01,90.52,69.79,87.26,7.17,3.0,1.06,244750.69,44.12,...,3.0,0.00,111.61,12.93,36.46,62.53,3352397.25,2.893069e+07,42.11,0.000050
2481,Prizy-ZV,2022-10-01,90.52,69.79,87.26,7.17,3.0,1.09,270900.52,44.12,...,5.0,6.17,81.65,10.59,36.46,62.53,3130631.76,2.412814e+07,42.11,0.000042
2482,Prizy-ZV,2022-11-01,90.52,69.79,87.26,7.17,3.0,1.15,190178.45,44.12,...,2.0,0.00,90.51,11.98,36.46,62.53,2609846.67,1.972099e+07,42.11,0.000034


In [98]:
# Now that we created our Market Share metric lets visualize its correlation with other metrics
market_share_corr = (
    pivoted.drop(["Company-Brand", "Dates"], axis=1)
    .corr()["Market Share"]
    .sort_values(ascending=False)
)

fig = px.bar(
    y=market_share_corr.index,
    x=market_share_corr.values,
    orientation="h",
    title="Correlation with Market Share",
    labels={"x": "Correlation", "y": "Metrics"},
    height=800,
)
fig.show()

In [99]:
def plot_all_metrics(df2, plot_only_prizy=False, save_plots=False, save_path="plots/all_company_metrics", show_plots=True, separate_prizy_x_y=True):
    # create directory if it doesn't exist
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    df2["Company"] = df2["Company-Brand"].apply(lambda x: x.split("-")[0])
    df2["Brand"] = df2["Company-Brand"].apply(lambda x: x.split("-")[-1])

    if separate_prizy_x_y:
        y_brands = [
            "Y",
            "Y-i",
            "Y-ii",
            "Y-iii",
        ]

        x_brands = [
            "X",
            "X-i",
            "X-ii",
        ]
        df2.loc[df2['Brand'].isin(y_brands), 'Company'] = "Prizy_y"
        df2.loc[df2['Brand'].isin(x_brands), 'Company'] = "Prizy_x"

    company_means = df2.groupby(["Company", "Metric", "Dates"])["Value"].mean()
    company_means = company_means.reset_index()

    for metric in company_means["Metric"].unique():
        metric_mean = company_means[company_means["Metric"] == metric]["Value"].mean()

        if plot_only_prizy:
            company_means = company_means[company_means["Company"].isin(["Prizy_x", "Prizy_y", "Prizy"])]

        fig = px.line(company_means[company_means["Metric"] == metric], x="Dates", y="Value", color="Company")
        fig.add_trace(go.Scatter(x=company_means[company_means["Metric"] == metric]["Dates"], y=[metric_mean] * len(company_means[company_means["Metric"] == metric]), mode="lines", name="Mean", line=dict(color="red", width=3, dash="dash")))
        

        fig.update_layout(title=metric, legend=dict(x=1.05, y=1, bordercolor="gray", borderwidth=1))

        if save_plots:
            os.makedirs(save_path, exist_ok=True)
            metric = re.sub(r'\([^)]*\)', '', metric).strip()
            fig.write_image(f"{save_path}/{metric}.png")

        if show_plots:
            fig.update_xaxes(tickangle=45)
            fig.show()



plot_all_metrics(df2, plot_only_prizy=True, save_plots=True, save_path="plots/all_company_metrics", show_plots=False, separate_prizy_x_y=True)


In [100]:

prizy_y = df2[df2["Company"] == "Prizy_y"]
prizy_x = df2[df2["Company"] == "Prizy_x"]

prizy_y_pivot = prizy_y.pivot_table(index='Dates', columns='Metric', values='Value')
prizy_x_pivot = prizy_x.pivot_table(index='Dates', columns='Metric', values='Value')


# Define a function to forecast a given metric for a given company
def forecast_metric(df, metric):
    # Create a new dataframe with the required columns
    df = df.copy()
    df = df.reset_index()
    data = df[['Dates', metric]].rename(columns={metric: 'y'})
    data = data.dropna()  # Drop rows with missing values
    data["ds"] = data["Dates"]
    # Create and fit a new Prophet model
    model = Prophet()
    model.fit(data)

    future = model.make_future_dataframe(periods=6*12, freq='M')  # Forecast for 6 years
    forecast = model.predict(future)
    forecast = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

    # Rename the columns to match the original dataframe
    forecast = forecast.rename(
        columns={
            'ds': 'date',
            'yhat': metric,
            'yhat_lower': f'{metric}_lower',
            'yhat_upper': f'{metric}_upper',
        }
    )

    # Set the index to be the date and return the forecast dataframe
    return forecast.set_index('date')


# Create an empty dataframe to store the forecasts
prizy_x_predictions = pd.DataFrame()
prizy_y_predictions = pd.DataFrame()

# Loop through each metric and forecast it for Prizy-X
for metric in prizy_x_pivot.columns:
    metric_predictions = forecast_metric(prizy_x_pivot, metric)
    prizy_x_predictions = pd.concat([prizy_x_predictions, metric_predictions], axis=1)

    metric_predictions = forecast_metric(prizy_y_pivot, metric)
    prizy_y_predictions = pd.concat([prizy_y_predictions, metric_predictions], axis=1)



In [101]:

def plot_forecast(pivots: tuple, predictions: tuple, save_plots=False, save_path="plots/forecast", show_plots=True):    
    # in predictions cut all values before 2022-12-31
    prizy_x_pivot, prizy_y_pivot = pivots
    prizy_x_predictions, prizy_y_predictions = predictions

    prizy_x_predictions = prizy_x_predictions[prizy_x_predictions.index > "2022-12-31"]
    prizy_y_predictions = prizy_y_predictions[prizy_y_predictions.index > "2022-12-31"]

    for metric in prizy_x_pivot.columns:
        fig, ax = plt.subplots(figsize=(15, 5))
        
        # metric, metric_lower, metric_upper 
        # and their corresponding prizy_x and prizy_y

        # Plot the actual values
        sns.lineplot(data=prizy_x_pivot, x=prizy_x_pivot.index, y=metric, ax=ax, label="Prizy-X", color="red")
        sns.lineplot(data=prizy_y_pivot, x=prizy_y_pivot.index, y=metric, ax=ax, label="Prizy-Y", color="blue")

        # Plot the forecasts
        sns.lineplot(data=prizy_x_predictions, x=prizy_x_predictions.index, y=metric, ax=ax, label="Prizy-X Forecast", color="red")
        sns.lineplot(data=prizy_y_predictions, x=prizy_y_predictions.index, y=metric, ax=ax, label="Prizy-Y Forecast", color="blue")

        # Plot the confidence intervals
        ax.fill_between(
            prizy_x_predictions.index,
            prizy_x_predictions[f'{metric}_lower'],
            prizy_x_predictions[f'{metric}_upper'],
            alpha=0.3,
            label="Confidence Interval",
            color="red"
        )

        ax.fill_between(
            prizy_y_predictions.index,
            prizy_y_predictions[f'{metric}_lower'],
            prizy_y_predictions[f'{metric}_upper'],
            alpha=0.3,
            label="Confidence Interval",
            color="blue"
        )


        plt.title(metric)

        if save_plots:
            os.makedirs(save_path, exist_ok=True)
            metric = re.sub(r'\([^)]*\)', '', metric).strip()
            fig.savefig(f"{save_path}/{metric}.png")

        if show_plots:
            plt.show()
        plt.close()

plot_forecast((prizy_x_pivot, prizy_y_pivot), (prizy_x_predictions, prizy_y_predictions), save_plots=True, save_path="plots/forecast", show_plots=False)